<a href="https://colab.research.google.com/github/jhonda18/Python3/blob/main/Clase_9_20210619.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción al Machine Learning

En los últimos años, el aprendizaje automático se ha dado a conocer por sus grandes logros, un ejemplo de esto, es el tan conocido clasificador de *Spam*. Aunque el aprendizaje automático ha cogido mucha fuerza en los últimos años, muchos de los algoritmos ya existían desde hace mucho tiempo.

Algo muy común es creer que el aprendizaje automático es Inteligencia Artificial, si bien es una rama de la inteligencia artificial, y existe una retroalimentación constante entre ML e IA, no todas las aplicaciones de ML terminan en IA.

**¿Que es Machine Learning?**

El Machine Learning es la ciencia (y el arte) de programar ordenadores para que aprendan a partir de los datos. Algunas definiciones:

* El ML es el campo de estudio que da a los ordenadores la capacidad de aprender sin ser programados de manera explícita. (**Arthur Samuel, 1959**)
* Se dice que un programa de ordenador aprende de la experiencia **E**, con respecto a una tarea **T** y una medida de rendimiento **R**, si su rendimiento en **T**, medido por **R**, mejora con la experiencia **E**. (**Tom Mitchell, 1997**)

## Tipos de aprendizaje

* Aprendizaje supervisado
    * Clasificación.
        * Clasificación binaria.
        * Clasificación multiclase.
    * Regresión.

* Aprendizaje no supervisado.
    * Clustering.
    * Reducción de la dimensionalidad.
    * Detección de anomalías.

* Aprendizaje semi-supervisado
* Aprendizaje por refuerzo

### Aprendizaje supervisado

Los algoritmos de aprendizaje supervisado están diseñados para aprender mediante ejemplos con sus respectivas respuestas. Contamos con datos de entrada, en general de forma estructurada, es decir, tenemos muchas observaciones con columnas (variables) y dentro de esos datos, existe una variable que queremos predecir. Por ejemplo, dadas ciertas características de mediciones queremos predecir si una persona tiene diabetes o no.

El flujo sel aprendizaje supervisado es así:

* Tomamos nuestros datos y separamos en variables independientes (predictoras) ***X***, y en una variable ***y*** que queremos predecir (variable dependiente).
* Mostramos pares *(x, y)* a un algoritmo preparado para aprender de nuestros datos, de forma tal que crea un conjunto de reglas o asociaciones para, dada una entrada ***x***, predecir ***y***.
* Cuando el modelo está entranado, queremos que el modelo haga una predicción sobre datos no observados.

Cuando nuestra variable de interés es una categoría, significa que tenemos un problema de **clasificación**. Si nuestra variable de interés es una variable numérica continua, tenemos un problema de **regresión**.

#### Regresión

Wikipedia define el análisis de regresión como:

En estadística, el análisis de regresión es un proceso estadístico para estimar las relaciones entre variables. Incluye muchas técnicas para el modelado y análisis de diversas variables, cuando la atención se centra en la relación entre una variable dependiente y una o más variables independientes (o predictoras).

In [ ]:
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

pd.set_option("float_format", "{:.2f}".format)
warnings.filterwarnings(action="ignore")

In [ ]:
auto = pd.read_csv("https://raw.githubusercontent.com/stivenlopezg/Modulo-Python-3/master/data/mpg.csv", dtype={"model_year": "category"})
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.00,8,307.00,130.00,3504,12.00,70,usa,chevrolet chevelle malibu
1,15.00,8,350.00,165.00,3693,11.50,70,usa,buick skylark 320
2,18.00,8,318.00,150.00,3436,11.00,70,usa,plymouth satellite
3,16.00,8,304.00,150.00,3433,12.00,70,usa,amc rebel sst
4,17.00,8,302.00,140.00,3449,10.50,70,usa,ford torino


In [ ]:
mpg.describe()

count   398.00
mean     23.51
std       7.82
min       9.00
25%      17.50
50%      23.00
75%      29.00
max      46.60
Name: mpg, dtype: float64

In [ ]:
indices = auto.sample(n=4, random_state=42).index

auto.loc[indices, "origin"] = np.nan

In [ ]:
mpg = auto.pop("mpg")

train_data, test_data, train_mpg, test_mpg = train_test_split(auto, mpg,
                                                              train_size=0.7)

In [ ]:
test_data, new_data, test_mpg, new_mpg = train_test_split(test_data, test_mpg,
                                                          test_size=0.4)

In [ ]:
# new_data.to_csv("mpg_new_data.csv", index=False, sep=",")

In [ ]:
# new_mpg.to_csv("mpg_label.csv", index=False, sep=',')

##### Vecinos más cercanos (K-NN)

Es un modelo basado en una idea sencilla. Imagina que tienes una instancia (observación) de la cual quieres saber su etiqueta; entonces, simplemente lo que necesitas hacer es buscar la instancia más parecida (más cercana) que esté etiquetada y asignarle a la instancia la etiqueta de esa instancia. Es decir, la etiqueta que asigna vecinos más cercanos es la etiqueta de la instancia más parecida, lo que llamamos el vecino más cercano.

**¿Cómo medimos la cercanía?**

Para medir la cercanía usamos distancias, y cada observación es un punto en un espacio n-dimensional.



In [ ]:
cols_to_drop = ["model_year", "name"]

train_data.drop(labels=cols_to_drop, axis=1, inplace=True)
test_data.drop(labels=cols_to_drop, axis=1, inplace=True)

In [ ]:
numerical_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration"]
categorical_features = ["origin"]

In [ ]:
imputer_num = SimpleImputer(strategy="median").fit(train_data[numerical_features])

imputer_cat = SimpleImputer(strategy="most_frequent").fit(train_data[categorical_features])

train_data.loc[:, numerical_features] = imputer_num.transform(train_data[numerical_features])
train_data.loc[:, categorical_features] = imputer_cat.transform(train_data[categorical_features])

In [ ]:
scaler = StandardScaler().fit(train_data[numerical_features])

train_data.loc[:, numerical_features] = scaler.transform(train_data[numerical_features])

In [ ]:
train_data.head()

,cylinders,displacement,horsepower,weight,acceleration,origin
0,1.52,1.06,0.64,0.59,-1.32,usa
283,0.34,0.35,-0.38,0.31,0.89,usa
97,0.34,0.29,0.00,0.15,0.29,usa
73,1.52,1.06,0.64,1.28,-0.61,usa
357,-0.84,-0.71,-0.13,-0.44,-0.32,japan


In [ ]:
ohe = OneHotEncoder(handle_unknown="error", drop="first", sparse=False).fit(train_data[categorical_features])

In [ ]:
cat_columns = [i.split("_")[1] for i in  ohe.get_feature_names().tolist()]

In [ ]:
ohe_df = pd.DataFrame(ohe.transform(train_data[categorical_features]),
                      columns=cat_columns)
ohe_df.head()

,japan,usa
0,0.00,1.00
1,0.00,1.00
2,0.00,1.00
3,0.00,1.00
4,1.00,0.00


In [ ]:
# Otra forma

# ohe_df_2 = pd.DataFrame(ohe.transform(train_data[categorical_features]),
                        # columns=cat_columns, index=train_data.index)

# ohe_df_2

In [ ]:
train_data.drop(labels=categorical_features, axis=1, inplace=True)

train_data = pd.concat(objs=[train_data.reset_index(drop=True),
                             ohe_df], axis=1)

train_data.head()

,cylinders,displacement,horsepower,weight,acceleration,japan,usa
0,1.52,1.06,0.64,0.59,-1.32,0.00,1.00
1,0.34,0.35,-0.38,0.31,0.89,0.00,1.00
2,0.34,0.29,0.00,0.15,0.29,0.00,1.00
3,1.52,1.06,0.64,1.28,-0.61,0.00,1.00
4,-0.84,-0.71,-0.13,-0.44,-0.32,1.00,0.00


In [ ]:
knn = KNeighborsRegressor(n_neighbors=5).fit(train_data, train_mpg)


print(f"El r2 en entrenamiento es: {round(knn.score(train_data, train_mpg), 2)}")

El r2 en entrenamiento es: 0.81



##### ¿Cómo medimos el desempeño de nuestro modelo?

Necesitamos una medida de qué tan bien (o mal) el modelo se ajusta a los datos de entrenamiento. Esta medida de evaluación (función de costo) es el error calculado entre la recta generada $\hat{y}$ (o el hiperplano) a los puntos reales. En el entrenamiento del modelo se encuentran los valores de $w_i$ que minimicen dicha función de costo. Entre las métricas más populares encontramos:

* Error medio absoluto (MAE)

$$MAE = \frac{1}{m}\sum_{i=1}^{m}|\hat{y}_i -y_i|$$

* Error cuadrático medio (MSE)

$$MSE=\frac{1}{m}\sum_{i=1}^{m}\left(\hat{y}_i -y_i\right)^2$$

* Raíz del error cuadrático medio (RMSE)

$$RMSE=\sqrt{\frac{1}{m}\sum_{i=1}^{m}\left(\hat{y}({\bf x})_i -y_i\right)^2}$$

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
class RegressionEvaluator:
  def __init__(self, observed: pd.Series or list, predicted: pd.Series or list):
    self.observed = observed
    self.predicted = predicted
    self.metrics = None

  def calculate_metrics(self):
    self.metrics = {
        "mae": round(mean_absolute_error(y_true=self.observed, y_pred=self.predicted), 2),
        "mse": round(mean_squared_error(y_true=self.observed, y_pred=self.predicted), 2),
        "r2": round(r2_score(y_true=self.observed, y_pred=self.predicted), 2)
    }
    return self

  def print_metrics(self):
    print(f"El MAE es: {self.metrics['mae']}")
    print(f"El MSE es: {self.metrics['mse']}")
    print(f"El RMSE es: {round(np.sqrt(self.metrics['mse']), 2)}")
    print(f"El R2 es: {self.metrics['r2']}")

In [ ]:
train_data.head()

,cylinders,displacement,horsepower,weight,acceleration,japan,usa
0,1.52,1.06,0.64,0.59,-1.32,0.00,1.00
1,0.34,0.35,-0.38,0.31,0.89,0.00,1.00
2,0.34,0.29,0.00,0.15,0.29,0.00,1.00
3,1.52,1.06,0.64,1.28,-0.61,0.00,1.00
4,-0.84,-0.71,-0.13,-0.44,-0.32,1.00,0.00


In [ ]:
test_data.head()

,cylinders,displacement,horsepower,weight,acceleration,origin
298,8,350.00,125.00,3900,17.40,usa
220,4,85.00,70.00,1945,16.80,japan
10,8,383.00,170.00,3563,10.00,usa
198,4,91.00,53.00,1795,17.40,NaN
17,6,200.00,85.00,2587,16.00,usa


In [ ]:
# Imputamos datos numéricos

test_data.loc[:, numerical_features] = imputer_num.transform(test_data[numerical_features])

# Escalamos datos numéricos

test_data.loc[:, numerical_features] = scaler.transform(test_data[numerical_features])

# Imputamos datos categóricos

test_data.loc[:, categorical_features] = imputer_cat.transform(test_data[categorical_features])

# Codificar datos categóricos


test_data = pd.concat(objs=[test_data.reset_index(drop=True),
                            pd.DataFrame(ohe.transform(test_data[categorical_features]),
                                                       columns=cat_columns)],
                       axis=1)

test_data.drop(labels=categorical_features, axis=1, inplace=True)

test_data.head()

,cylinders,displacement,horsepower,weight,acceleration,japan,usa
0,1.52,1.46,0.51,1.05,0.61,0.00,1.00
1,-0.84,-1.03,-0.89,-1.22,0.39,1.00,0.00
2,1.52,1.77,1.66,0.66,-2.04,0.00,1.00
3,-0.84,-0.97,-1.33,-1.40,0.61,0.00,1.00
4,0.34,0.05,-0.51,-0.48,0.11,0.00,1.00


In [ ]:
evaluation = RegressionEvaluator(observed=test_mpg, predicted=knn.predict(test_data))
evaluation.calculate_metrics()
evaluation.print_metrics()

El MAE es: 2.67
El MSE es: 12.61
El RMSE es: 3.55
El R2 es: 0.81


In [ ]:
linear_reg = LinearRegression().fit(train_data, train_mpg)

evaluation_reg = RegressionEvaluator(observed=test_mpg, predicted=linear_reg.predict(test_data))
evaluation_reg.calculate_metrics()
evaluation_reg.print_metrics()

El MAE es: 3.3
El MSE es: 17.9
El RMSE es: 4.23
El R2 es: 0.73


##### Pipeline

Una secuencia de datos que procesan componentes se llama *pipeline* de datos. Los *pipelines* son muy comunes en los sistemas de machine learning, puesto que hay muchos datos que manipular y muchas transformaciones de datos que aplicar.


In [ ]:
auto = pd.read_csv("https://raw.githubusercontent.com/stivenlopezg/Modulo-Python-3/master/data/mpg.csv", dtype={"model_year": "category"})
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.00,8,307.00,130.00,3504,12.00,70,usa,chevrolet chevelle malibu
1,15.00,8,350.00,165.00,3693,11.50,70,usa,buick skylark 320
2,18.00,8,318.00,150.00,3436,11.00,70,usa,plymouth satellite
3,16.00,8,304.00,150.00,3433,12.00,70,usa,amc rebel sst
4,17.00,8,302.00,140.00,3449,10.50,70,usa,ford torino


In [ ]:
mpg = auto.pop("mpg")

train_data, test_data, train_mpg, test_mpg = train_test_split(auto, mpg,
                                                              train_size=0.7)

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

In [ ]:
numeric_preprocessing = Pipeline(steps=[("num_imputer", SimpleImputer(strategy="median")),
                                        ("st_scaler", StandardScaler())])

In [ ]:
numeric_preprocessing

Pipeline(memory=None,
         steps=[('num_imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('st_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True))],
         verbose=False)

In [ ]:
numeric_preprocessing = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
numeric_preprocessing

Pipeline(memory=None,
         steps=[('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True))],
         verbose=False)

In [ ]:
categoric_preprocessing = make_pipeline(SimpleImputer(strategy="most_frequent"),
                                        OneHotEncoder(handle_unknown="error",
                                                      drop="first", sparse=False))

In [ ]:
preprocessor = ColumnTransformer(transformers=[("numeric_preprocessing", numeric_preprocessing, numerical_features),
                                               ("categoric_preprocessing", categoric_preprocessing, categorical_features)],
                                 remainder="drop")

In [ ]:
preprocessor = make_column_transformer((numeric_preprocessing, numerical_features),
                                       (categoric_preprocessing, categorical_features), remainder="drop")

preprocessor

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('pipeline-1',
                                 Pipeline(memory=None,
                                          steps=[('simpleimputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0)),
                                                 ('standardscaler',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
      

In [ ]:
knn = make_pipeline(preprocessor, KNeighborsRegressor(n_neighbors=5)).fit(train_data, train_mpg)

evaluation = RegressionEvaluator(observed=test_mpg, predicted=knn.predict(test_data))
evaluation.calculate_metrics()
evaluation.print_metrics()

El MAE es: 2.86
El MSE es: 17.42
El RMSE es: 4.17
El R2 es: 0.7


In [ ]:
test_data.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
198,4,91.00,53.00,1795,17.40,76,japan,honda civic
101,6,198.00,95.00,2904,16.00,73,usa,plymouth duster
374,4,151.00,nan,3035,20.50,82,usa,amc concord dl
148,4,116.00,75.00,2246,14.00,74,europe,fiat 124 tc
235,4,97.00,75.00,2265,18.20,77,japan,toyota corolla liftback


In [ ]:
joblib.dump(knn, filename="knn.joblib")

['knn.joblib']

In [ ]:
knn = None

## Emular sobre predicciones nuevas

In [ ]:
data = pd.read_csv("mpg_new_data.csv")
data.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,6,250.00,110.00,3645,16.20,76,usa,pontiac ventura sj
1,4,107.00,90.00,2430,14.50,70,europe,audi 100 ls
2,4,135.00,84.00,2490,15.70,81,usa,plymouth reliant
3,4,91.00,70.00,1955,20.50,71,usa,plymouth cricket
4,4,146.00,67.00,3250,21.80,80,europe,mercedes-benz 240d


In [ ]:
# 

imputer_num

imputer_cat

In [ ]:
model = joblib.load(filename="knn.joblib")
model

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline-1',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                           

In [ ]:
data["mpg"] = model.predict(data)

data

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,mpg
0,6,250.00,110.00,3645,16.20,76,usa,pontiac ventura sj,19.50
1,4,107.00,90.00,2430,14.50,70,europe,audi 100 ls,26.00
2,4,135.00,84.00,2490,15.70,81,usa,plymouth reliant,27.32
3,4,91.00,70.00,1955,20.50,71,usa,plymouth cricket,23.90
4,4,146.00,67.00,3250,21.80,80,europe,mercedes-benz 240d,25.98
5,8,350.00,105.00,3725,19.00,81,usa,oldsmobile cutlass ls,20.22
6,6,200.00,81.00,3012,17.60,76,usa,ford maverick,21.50
7,6,250.00,88.00,3139,14.50,71,usa,ford mustang,19.60
8,4,91.00,67.00,1965,15.70,82,japan,honda civic (auto),35.86
9,4,98.00,79.00,2255,17.70,76,usa,dodge colt,25.00
